> - #### **Условие**
> #### **1.** Cоединить 5  процессов, и их нити (в каждом по три) методом Pipe, Queue, предварительно добавив еще один фильтр.  Примените один дуплексный канал.
> #### **2.** Представить фоновый процесс (Log)  как Server, а  остальные процессы организовать в 2 клиента (Client). Реализовать их соединение методом сокетов.
> #### **3.**  Реализовать вывод на экран: 
> - #### начало/окончание, номер - нити/процесса  
> - #### вывод пересылаемых данных
 

> #### **Фильтры**
> #### **1.**  Разработать фильтр, подсчитывающий количество слов в каждой строке вводимого текста. Количество слов выводится перед строкой и отделяется двоеточием.
> #### **2.** Разработать фильтр, удаляющий избыточные пробелы из входного текста.

In [ ]:
import socket
from threading import current_thread
from multiprocessing import current_process
from threading import Thread
from multiprocessing import Queue
import socket
import random
import time
import logging
from multiprocessing import Process
from multiprocessing import Pipe
import string

HOST = "127.0.0.1" 
PORT_1 = 65327 
PORT_2 = 65333

def randomword(length):
   letters = string.ascii_lowercase
   return ',   '.join(random.choice(letters) for i in range(length))

def thread(HOST,PORT,queue_s):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        conn, addr = s.accept()
        with conn:
            data = ' '
            while data:
                data = conn.recv(4096)
                if data:
                    queue_s.put(f"{data}")

def server(port_1, port_2):
    queue_s = Queue()
    logger = logging.getLogger('app')
    logger.setLevel(logging.DEBUG)
    logger.addHandler(logging.StreamHandler())
    thread1=Thread(target = thread, args = ([HOST, port_1, queue_s]))
    thread2=Thread(target = thread, args=([HOST, port_2, queue_s]))
    thread1.start()
    thread2.start()
    thread1.join()
    thread2.join()
    while queue_s.empty() != True:
        item = queue_s.get()
        item = item[2:len(item) - 1:]
        item_spl = item.split('|')
        for i in range(len(item_spl)):
            logger.info(item_spl[i])

def thread_task(strok,queue_s, input,output, timequeue, number, task):
    start = time.time()
    if input != 0:
        strok = input.recv()
    strbuff = ''
    time.sleep(0.01)
    strspl = strok.split()
    if task == 1:
        strbuff = strok.replace("   ", " ")
    else:
        strbuff = (str(len(strspl)) + ":" + " " + strok)
    thread_name = current_thread().name
    process_name = current_process().name
    queue_s.put(f'Thread {thread_name} in process {process_name} done with result {strbuff} (started with string {strok})')
    output.send(strbuff)
    queue_s.put(f'Sent {strbuff} to next thread')
    tn = thread_name.split()
    thread_name = tn[0]
    s_start = time.time()
    time.sleep(0.0001*random.random())
    s_finish = time.time()
    time.sleep(0.01)
    end = time.time()
    timequeue.put(f'{start}, {end}, C{number} {process_name} {thread_name}, {s_start}, {s_finish}')      

def process_task(queue_s, number, timequeue):
    start = time.time()
    conn1, conn2 = Pipe(duplex = True)
    conn3, conn4 = Pipe()
    conn5, conn6 = Pipe()
    number = random.randint(1, 10)
    stringa = randomword(number)
    thread1 = Thread(target=thread_task, args=([stringa, queue_s, 0 , conn1, timequeue, number, 1]))
    thread2 = Thread(target=thread_task, args=(['', queue_s, conn2, conn3, timequeue, number, 2])) 
    thread3 = Thread(target=thread_task, args=(['', queue_s, conn4, conn5, timequeue, number, 1]))
    thread1.start()
    thread2.start()
    thread3.start()
    thread1.join()
    s_start = time.time()
    time.sleep(0.01 * random.random())
    s_finish = time.time()
    thread2.join()
    thread3.join()
    item = conn6.recv()
    queue_s.put(f'Finally {item}')
    process_name = current_process().name
    queue_s.put(f'Process {process_name} done.')
    end = time.time()
    timequeue.put(f'{start}, {end}, C{number} {process_name}, {s_start}, {s_finish}')

def client(port, number, timequeue):
    queue_s = Queue()
    msg = b''
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, port))
        item = f'Client {number} started'
        b = bytes(item, encoding = 'utf-8')
        s.sendall(b)
        processes = [Process(target = process_task, args = (queue_s,number,timequeue)) for i in range(number + 1)]
        for process in processes:
            process.start()
        for process in processes:
            process.join()
        while queue_s.empty() != True:
            item = queue_s.get()
            b = bytes(item, encoding = 'utf-8')
            msg = msg + b'|' + b
        item = f'Client {number} done.'
        b = bytes(item, encoding = 'utf-8')
        msg = msg + b'|' + b
        s.sendall(msg)

if __name__ == '__main__':
    timequeue = Queue()
    logger_p = Process(target = server, args=(PORT_1,PORT_2), daemon = True)
    logger_p.start()
    client_1 = Process(target = client, args = (PORT_1, 1, timequeue))
    client_2 = Process(target = client, args = (PORT_2, 2, timequeue))
    client_1.start()
    client_2.start()
    client_1.join()
    client_2.join()
    logger_p.join()

> - #### **Условие**
> #### **1.** Визуализировать анимацией выполненние нитей и процессов на временной оси из предыдущей работы

 

> - #### **Решение**

In [ ]:
import socket
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from threading import current_thread
from multiprocessing import current_process
from threading import Thread
from multiprocessing import Queue
import socket
import random
import time
import logging
from multiprocessing import Process
from multiprocessing import Pipe
import string

HOST = "127.0.0.1" 
PORT_1 = 65327 
PORT_2 = 65333

def randomword(length):
   letters = string.ascii_lowercase
   return ', '.join(random.choice(letters) for i in range(length))

def thread(HOST,PORT,queue_s):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        conn, addr = s.accept()
        with conn:
            data = ' '
            while data:
                data = conn.recv(4096)
                if data:
                    queue_s.put(f"{data}")

def server(port_1, port_2):
    queue_s = Queue()
    logger = logging.getLogger('app')
    logger.setLevel(logging.DEBUG)
    logger.addHandler(logging.StreamHandler())
    thread1=Thread(target = thread, args = ([HOST, port_1, queue_s]))
    thread2=Thread(target = thread, args=([HOST, port_2, queue_s]))
    thread1.start()
    thread2.start()
    thread1.join()
    thread2.join()
    while queue_s.empty() != True:
        item = queue_s.get()
        item = item[2:len(item) - 1:]
        item_spl = item.split('|')
        for i in range(len(item_spl)):
            logger.info(item_spl[i])

def thread_task(strok,queue_s, input,output, timequeue, number, task):
    start = time.time()
    if input != 0:
        strok = input.recv()
    strbuff = ''
    time.sleep(0.001)
    strspl = strok.split()
    if task == 1:
        strbuff = strok.replace("b", "##")
    else:
        strbuff = (str(len(strspl)) + ":" + " " + strok)
    thread_name = current_thread().name
    process_name = current_process().name
    queue_s.put(f'Thread {thread_name} in process {process_name} done with result {strbuff} (started with string {strok})')
    output.send(strbuff)
    queue_s.put(f'Sent {strbuff} to next thread')
    tn = thread_name.split()
    thread_name = tn[0]
    s_start = time.time()
    time.sleep(0.001*random.random())
    s_finish = time.time()
    time.sleep(0.001)
    end = time.time()
    timequeue.put(f'{start}, {end}, C{number} {process_name} {thread_name}, {s_start}, {s_finish}')      

def process_task(queue_s, number, timequeue):
    start = time.time()
    conn1, conn2 = Pipe(duplex = True)
    conn3, conn4 = Pipe()
    conn5, conn6 = Pipe()
    number = random.randint(1, 10)
    stringa = randomword(number)
    thread1 = Thread(target=thread_task, args=([stringa, queue_s, 0 , conn1, timequeue, number, 1]))
    thread2 = Thread(target=thread_task, args=(['', queue_s, conn2, conn3, timequeue, number, 2])) 
    thread3 = Thread(target=thread_task, args=(['', queue_s, conn4, conn5, timequeue, number, 1]))
    thread1.start()
    thread2.start()
    thread3.start()
    thread1.join()
    s_start = time.time()
    time.sleep(0.001 * random.random())
    s_finish = time.time()
    thread2.join()
    thread3.join()
    item = conn6.recv()
    queue_s.put(f'Finally {item}')
    process_name = current_process().name
    queue_s.put(f'Process {process_name} done.')
    end = time.time()
    timequeue.put(f'{start}, {end}, C{number} {process_name}, {s_start}, {s_finish}')

def client(port, number, timequeue):
    queue_s = Queue()
    msg = b''
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, port))
        item = f'Client {number} started'
        b = bytes(item, encoding = 'utf-8')
        s.sendall(b)
        processes = [Process(target = process_task, args = (queue_s,number,timequeue)) for i in range(number + 1)]
        for process in processes:
            process.start()
        for process in processes:
            process.join()
        while queue_s.empty() != True:
            item = queue_s.get()
            b = bytes(item, encoding = 'utf-8')
            msg = msg + b'|' + b
        item = f'Client {number} done.'
        b = bytes(item, encoding = 'utf-8')
        msg = msg + b'|' + b
        s.sendall(msg)

if __name__ == '__main__':
    timequeue = Queue()
    logger_p = Process(target = server, args=(PORT_1,PORT_2), daemon = True)
    logger_p.start()
    client_1 = Process(target = client, args = (PORT_1, 1, timequeue))
    client_2 = Process(target = client, args = (PORT_2, 2, timequeue))
    client_1.start()
    client_2.start()
    client_1.join()
    client_2.join()
    logger_p.join()

    start_times = []
    finish_times = []
    names = []
    s_sleep_times = []
    f_sleep_times = []
    start = 100000000000.0
    i = 0
    eps = 0.00001
    while timequeue.empty() != True:
        item = timequeue.get()
        it = item.split(',')
        if len(it) == 3:
            start_times.append(float(it[0]))
            finish_times.append(float(it[1]))
            names.append(it[2])
            s_sleep_times.append(-1)
            f_sleep_times.append(-1)
        else:
            start_times.append(float(it[0]))
            finish_times.append(float(it[1]))
            names.append(it[2])
            s_sleep_times.append(float(it[3]))
            f_sleep_times.append(float(it[4]))
        if (start_times[i] - start) < eps:
            start = start_times[i]
        i += 1
    lst = []
    scale = 700
    max = round((max(finish_times) - start) * scale)
    for i in range(len(start_times)):
        start_times[i] = round((start_times[i] - start) * scale)
        finish_times[i] = round((finish_times[i] - start) * scale)
        s_sleep_times[i] = round((s_sleep_times[i] - start) * scale)
        f_sleep_times[i] = round((f_sleep_times[i] - start) * scale)
        lst.append([j if j < finish_times[i] else finish_times[i] for j in range(max)])
    plt.ion()
    fig = plt.figure(figsize = (7,5))
    axes = fig.add_subplot(1,1,1)
    axes.set_xlim(0, max + 20)
    colors = ['darkviolet','orangered','chartreuse']
    def animate(i):
        if i < max:
            for j in range(len(start_times)):
                if i >= start_times[j]:
                    if ( (i <= s_sleep_times[j]) | (s_sleep_times[j] < 0) ):
                        plt.barh(j, lst[j][i] - start_times[j], color = colors[j % 3], left = start_times[j])
                    else:
                        if ( (i > s_sleep_times[j]) & (i <= f_sleep_times[j]) ):
                            plt.barh(j, lst[j][i] - s_sleep_times[j], color = 'black', left = s_sleep_times[j])
                        else:
                            plt.barh(j, lst[j][i] - f_sleep_times[j], color = colors[j%3], left = f_sleep_times[j])
                else:
                    plt.barh(j, 0)

    plt.yticks(np.arange(len(start_times)), names)

    plt.title("Multithreading and multiprocessing", color = ("blue"))
    ani = FuncAnimation(fig, animate, interval = 1)

    plt.ioff()
    plt.show()